In [1]:
import pandas as pd 
url = "https://cocl.us/datascience_survey_data"
df = pd.read_csv(url, index_col=0) #load the first column as the index of the dataframe.
print(df.head())

                            Very interested  Somewhat interested  \
Big Data (Spark / Hadoop)              1332                  729   
Data Analysis / Statistics             1688                  444   
Data Journalism                         429                 1081   
Data Visualization                     1340                  734   
Deep Learning                          1263                  770   

                            Not interested  
Big Data (Spark / Hadoop)              127  
Data Analysis / Statistics              60  
Data Journalism                        610  
Data Visualization                     102  
Deep Learning                          136  


In [11]:
import pandas as pd
df = pd.read_csv("BL-Flickr-Images-Book.csv.csv")
print(df.head())
columns_to_drop = ['Edition Statement','Corporate Author', 'Corporate Contributors', 'Former owner', 
                   'Engraver', 'Contributors', 'Issuance type', 'Shelfmarks']
df = df.drop(columns_to_drop, axis=1, inplace=False)

# Set 'Identifier' as index
df = df.set_index('Identifier')
print("\nDataFrame shape:", df.shape)
print("\nColumn names:", df.columns.tolist())

   Identifier             Edition Statement      Place of Publication  \
0         206                           NaN                    London   
1         216                           NaN  London; Virtue & Yorston   
2         218                           NaN                    London   
3         472                           NaN                    London   
4         480  A new edition, revised, etc.                    London   

  Date of Publication              Publisher  \
0         1879 [1878]       S. Tinsley & Co.   
1                1868           Virtue & Co.   
2                1869  Bradbury, Evans & Co.   
3                1851          James Darling   
4                1857   Wertheim & Macintosh   

                                               Title     Author  \
0                  Walter Forbes. [A novel.] By A. A      A. A.   
1  All for Greed. [A novel. The dedication signed...  A., A. A.   
2  Love the Avenger. By the author of “All for Gr...  A., A. A.   
3  W

In [7]:
# Explore Date of Publication and Place of Publication columns
print("="*80)
print("EXPLORING DATE OF PUBLICATION")
print("="*80)
print(f"\nTotal records: {len(df)}")
print(f"Non-null dates: {df['Date of Publication'].notna().sum()}")
print(f"Null dates: {df['Date of Publication'].isna().sum()}")

print("\nSample Date of Publication values:")
print(df['Date of Publication'].value_counts().head(20))

print("\n" + "="*80)
print("EXPLORING PLACE OF PUBLICATION")
print("="*80)
print(f"Non-null places: {df['Place of Publication'].notna().sum()}")
print(f"Null places: {df['Place of Publication'].isna().sum()}")

print("\nSample Place of Publication values:")
print(df['Place of Publication'].value_counts().head(20))

EXPLORING DATE OF PUBLICATION

Total records: 8287
Non-null dates: 8106
Null dates: 181

Sample Date of Publication values:
Date of Publication
1897    157
1896    150
1893    130
1892    127
1898    125
1895    124
1899    120
1891    119
1894    118
1890    115
1889    113
1876    113
1888    105
1880    103
1887     99
1884     96
1886     93
1878     93
1869     92
1881     91
Name: count, dtype: int64

EXPLORING PLACE OF PUBLICATION
Non-null places: 8287
Null places: 0

Sample Place of Publication values:
Place of Publication
London            3868
Paris              479
Edinburgh          208
New York           177
Leipzig            119
Philadelphia        89
Berlin              70
Boston [Mass.]      52
Dublin              48
Glasgow             45
Oxford              44
Boston              41
Wien                38
Madrid              33
Stockholm           33
Bruxelles           28
Cambridge           26
Amsterdam           25
Stuttgart           24
Firenze             24
Nam

In [8]:
# CLEANING AND STANDARDIZING DATE OF PUBLICATION
print("="*80)
print("CLEANING DATE OF PUBLICATION")
print("="*80)

def clean_date_of_publication(date_str):
    """
    Extract the primary publication year from date strings.
    Handles formats like:
    - '1879' -> 1879
    - '1879 [1878]' -> 1879 (first year)
    - '[1879]' -> 1879
    - '1879?' -> 1879
    - NaN -> NaN
    """
    if pd.isna(date_str):
        return None
    
    date_str = str(date_str).strip()
    
    # Extract first 4-digit year found
    import re
    match = re.search(r'\d{4}', date_str)
    if match:
        return int(match.group())
    
    return None

# Apply cleaning
df['Date of Publication (cleaned)'] = df['Date of Publication'].apply(clean_date_of_publication)

print(f"\nSuccessfully cleaned {df['Date of Publication (cleaned)'].notna().sum()} date records")
print(f"\nCleaned Date of Publication (Year only):")
print(df['Date of Publication (cleaned)'].describe())
print(f"\nYear range: {df['Date of Publication (cleaned)'].min()} - {df['Date of Publication (cleaned)'].max()}")

# Show before/after samples
print("\nBefore/After Examples:")
sample_indices = df[df['Date of Publication'].notna()].index[:10]
comparison = pd.DataFrame({
    'Original': df.loc[sample_indices, 'Date of Publication'],
    'Cleaned': df.loc[sample_indices, 'Date of Publication (cleaned)']
})
print(comparison)

CLEANING DATE OF PUBLICATION

Successfully cleaned 8104 date records

Cleaned Date of Publication (Year only):
count    8104.000000
mean     1857.813425
std        41.551554
min      1510.000000
25%      1844.000000
50%      1868.000000
75%      1887.000000
max      1915.000000
Name: Date of Publication (cleaned), dtype: float64

Year range: 1510.0 - 1915.0

Before/After Examples:
               Original  Cleaned
Identifier                      
206         1879 [1878]   1879.0
216                1868   1868.0
218                1869   1869.0
472                1851   1851.0
480                1857   1857.0
481                1875   1875.0
519                1872   1872.0
874                1676   1676.0
1143               1679   1679.0
1280               1802   1802.0


In [9]:
# CLEANING AND STANDARDIZING PLACE OF PUBLICATION
print("\n" + "="*80)
print("CLEANING PLACE OF PUBLICATION")
print("="*80)

# Define mapping for city name standardization (non-English names and variants)
place_mapping = {
    'Wien': 'Vienna',
    'Bruxelles': 'Brussels',
    'Firenze': 'Florence',
    'Milano': 'Milan',
    'Napoli': 'Naples',
    'Torino': 'Turin',
    'Roma': 'Rome',
    'Venezia': 'Venice',
    'Gent': 'Ghent',
    'Den Haag': 'The Hague',
    'Köln': 'Cologne',
    'München': 'Munich',
    'Zürich': 'Zurich',
    'København': 'Copenhagen',
    'Lund': 'Lund',
    'Basel': 'Basel',
    'Bern': 'Bern',
    'Freiburg': 'Freiburg',
}

def clean_place_of_publication(place_str):
    """
    Standardize place of publication:
    1. Handle semicolon-separated multiple locations (take first)
    2. Remove state/province codes like [Mass.], [N.Y.]
    3. Standardize non-English city names
    4. Strip whitespace
    """
    if pd.isna(place_str):
        return None
    
    place_str = str(place_str).strip()
    
    # Split by semicolon and take first location
    if ';' in place_str:
        place_str = place_str.split(';')[0].strip()
    
    # Remove state/province codes in brackets
    import re
    place_str = re.sub(r'\s*\[.*?\]\s*', '', place_str).strip()
    
    # Apply standardization mapping
    for non_english, english in place_mapping.items():
        if place_str == non_english:
            place_str = english
    
    return place_str if place_str else None

# Apply cleaning
df['Place of Publication (cleaned)'] = df['Place of Publication'].apply(clean_place_of_publication)

print(f"\nSuccessfully cleaned {df['Place of Publication (cleaned)'].notna().sum()} place records")

print("\nCleaned Place of Publication - Top 20:")
print(df['Place of Publication (cleaned)'].value_counts().head(20))

# Show before/after for places with multiple values or state codes
print("\nBefore/After Examples (Places with Changes):")
sample_data = [
    ('London; Virtue & Yorston', 'London'),
    ('Boston [Mass.]', 'Boston'),
    ('Wien', 'Vienna'),
    ('Paris', 'Paris'),
    ('Bruxelles', 'Brussels'),
]

for original, expected in sample_data:
    actual = clean_place_of_publication(original)
    status = "✓" if actual == expected else "✗"
    print(f"{status} '{original}' -> '{actual}'")

print(f"\nTotal unique cleaned places: {df['Place of Publication (cleaned)'].nunique()}")


CLEANING PLACE OF PUBLICATION

Successfully cleaned 8287 place records

Cleaned Place of Publication - Top 20:
Place of Publication (cleaned)
London          3923
Paris            481
Edinburgh        211
New York         179
Leipzig          119
Boston            99
Philadelphia      90
Berlin            70
Dublin            48
Glasgow           45
Oxford            44
Vienna            39
Madrid            34
Cambridge         33
Stockholm         33
Brussels          29
Florence          26
Amsterdam         25
Chicago           24
Stuttgart         24
Name: count, dtype: int64

Before/After Examples (Places with Changes):
✓ 'London; Virtue & Yorston' -> 'London'
✓ 'Boston [Mass.]' -> 'Boston'
✓ 'Wien' -> 'Vienna'
✓ 'Paris' -> 'Paris'
✓ 'Bruxelles' -> 'Brussels'

Total unique cleaned places: 1361


In [10]:
# SUMMARY AND DATA QUALITY REPORT
print("\n" + "="*80)
print("DATA CLEANING SUMMARY")
print("="*80)

summary_data = {
    'Column': ['Date of Publication', 'Place of Publication'],
    'Original Non-null': [
        df['Date of Publication'].notna().sum(),
        df['Place of Publication'].notna().sum()
    ],
    'Cleaned Non-null': [
        df['Date of Publication (cleaned)'].notna().sum(),
        df['Place of Publication (cleaned)'].notna().sum()
    ],
    'Data Quality': [
        f"{(df['Date of Publication (cleaned)'].notna().sum() / len(df) * 100):.1f}%",
        f"{(df['Place of Publication (cleaned)'].notna().sum() / len(df) * 100):.1f}%"
    ],
    'Unique Values': [
        df['Date of Publication (cleaned)'].nunique(),
        df['Place of Publication (cleaned)'].nunique()
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n", summary_df.to_string(index=False))

print("\n" + "="*80)
print("CLEANED DATA SAMPLES")
print("="*80)

# Display sample records with both original and cleaned columns
sample_records = df[['Place of Publication', 'Place of Publication (cleaned)', 
                     'Date of Publication', 'Date of Publication (cleaned)']].head(15)
print("\n", sample_records.to_string())

print("\n" + "="*80)
print("TEMPORAL DISTRIBUTION (Cleaned Years)")
print("="*80)
print("\nDecade Distribution:")
df['Decade'] = (df['Date of Publication (cleaned)'] // 10 * 10).astype('Int64')
decade_dist = df['Decade'].value_counts().sort_index()
print(decade_dist)

print("\n" + "="*80)
print("GEOGRAPHIC DISTRIBUTION (Cleaned Places)")
print("="*80)
print("\nTop 15 Publishing Cities:")
print(df['Place of Publication (cleaned)'].value_counts().head(15).to_string())


DATA CLEANING SUMMARY

               Column  Original Non-null  Cleaned Non-null Data Quality  Unique Values
 Date of Publication               8106              8104        97.8%            255
Place of Publication               8287              8287       100.0%           1361

CLEANED DATA SAMPLES

                            Place of Publication       Place of Publication (cleaned) Date of Publication  Date of Publication (cleaned)
Identifier                                                                                                                             
206                                      London                               London         1879 [1878]                         1879.0
216                    London; Virtue & Yorston                               London                1868                         1868.0
218                                      London                               London                1869                         1869.0
472          